In [1]:
import tensorflow as tf

In [2]:
def create_generator(generator_inputs, generator_outputs_channels):
    # 512x512
    vgg16 = tf.contrib.keras.applications.VGG16(weights=None, include_top=False, input_tensor=generator_inputs)
    cut_model = tf.contrib.keras.models.Sequential(vgg16.layers[:-4])
    pool4 = cut_model.output
    def padding_conv(inputs, filters, padding, kernel_size):
        padding_tensor = tf.constant([[0,0],[padding[0],padding[0]],[padding[1],padding[1]],[0,0]])
        x = tf.pad(inputs, padding_tensor)
        return tf.layers.conv2d(x, filters, kernel_size, padding='valid')
    x1 = padding_conv(pool4, 512, [1, 1], [3, 3])
    x2 = padding_conv(pool4, 512, [1, 3], [3, 7])
    x3 = padding_conv(pool4, 512, [3, 1], [7, 3])
    x = x1+x2+x3
    
    pool5 = tf.layers.max_pooling2d(x, 2, 2)
    fully_convolution1 = tf.layers.conv2d(pool5, 512, 1)
    fully_convolution2 = tf.layers.conv2d(fully_convolution1, 512, 1)
    
    upsample = tf.layers.conv2d_transpose(fully_convolution2, 512, 4, 2, padding='same')
    fuse = tf.concat([pool4, upsample], axis=-1)
    logits = tf.layers.conv2d(fuse, generator_outputs_channels, 1, 1)
    outputs = tf.nn.sigmoid(logits)
    return logits, outputs

In [2]:
import tensorflow as tf
keras = tf.contrib.keras

In [3]:
input_tensor = keras.layers.Input(shape=[None,None,3])
vgg = keras.applications.VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)

In [4]:
import numpy as np

def get_kernel_size(factor):
    """
    Find the kernel size given the desired factor of upsampling.
    """
    return 2 * factor - factor % 2


def upsample_filt(size):
    """
    Make a 2D bilinear kernel suitable for upsampling of the given (h, w) size.
    """
    factor = (size + 1) // 2
    if size % 2 == 1:
        center = factor - 1
    else:
        center = factor - 0.5
    og = np.ogrid[:size, :size]
    return (1 - abs(og[0] - center) / factor) * \
           (1 - abs(og[1] - center) / factor)


def bilinear_upsample_weights(factor, number_of_classes):
    """
    Create weights matrix for transposed convolution with bilinear filter
    initialization.
    """
    
    filter_size = get_kernel_size(factor)
    
    weights = np.zeros((filter_size,
                        filter_size,
                        number_of_classes,
                        number_of_classes), dtype=np.float32)
    
    upsample_kernel = upsample_filt(filter_size)
    
    for i in range(number_of_classes):
        
        weights[:, :, i, i] = upsample_kernel
    
    return weights

In [5]:
upsample_weight_2 = tf.constant(bilinear_upsample_weights(2, 3))


In [6]:
image = tf.placeholder(tf.float32, shape=[None, None, None, 3])
image_shape = tf.shape(image)
output_upsample_shape = tf.stack([image_shape[0], 
                                  image_shape[1] * 2,
                                  image_shape[2] * 2, 
                                  image_shape[3]])
upsample_image = tf.nn.conv2d_transpose(image, upsample_weight_2, output_shape=output_upsample_shape, strides=[1,2,2,1])

In [12]:
import cv2 

path = '/Users/macos/Desktop/UNADJUSTEDNONRAW_thumb_1c7.jpg'
image_np = cv2.imread(path)
print(image_np.shape)
image_batch = np.expand_dims(image_np, 0)
print('im batch: ', image_batch.shape)
sess = tf.InteractiveSession()
rv = sess.run(upsample_image, {image:image_batch})
upsample_image_np = rv[0]
print('New shape:', upsample_image_np.shape)
cv2.imwrite('/Users/macos/Desktop/output_img.png', upsample_image_np)

(1024, 768, 3)
im batch:  (1, 1024, 768, 3)
New shape: (2048, 1536, 3)


True